In [ ]:
import pandas as pd
import ftfy

In [ ]:
# Lendo o dataframe dos emprestimos

df_emprestimos = pd.read_csv('../data/interim/emprestimos-concat.csv')

# Mostrando alguns emprestimos
# df_emprestimos.head()

In [ ]:
df_emprestimos['tipo_vinculo_usuario'].unique()

In [ ]:
# Lendo o dataframe com os emprestimos
df_exemplares = pd.read_parquet('../data/raw/dados_exemplares.parquet')

# Mostrando alguns exemplares
df_exemplares.head()

In [ ]:
# Renomeando as colunas codigo_barras para referenciar aos respectivos dataframes

df_emprestimos.rename(columns={'codigo_barras':'codigo_barras_emprestimo'}, inplace=True)

df_exemplares.rename(columns={'codigo_barras':'codigo_barras_exemplar'}, inplace=True)

In [ ]:
df_emprestimos.dtypes

In [ ]:
df_exemplares.columns

In [ ]:
# Unindo os dataframes

df = pd.merge(df_emprestimos, df_exemplares, left_on='codigo_barras_emprestimo', right_on='codigo_barras_exemplar')

In [ ]:
# Mostrando alguns registros do dataframe unido
df.head()

In [ ]:
# Retirando colunas inúteis (registro_sistema)
df.drop(columns=['registro_sistema'], inplace=True)

In [ ]:
df.columns

Os itens do acervo em uma biblioteca são organizados por um sistema de classificação de acordo com o respectivo tema. Existem diversos sistemas, mas este conjunto está de acordo com a CDU - Classificação Decimal Universal. Esta classificação é decimal, pois varia de acordo com a classe de cada assunto:


000 a 099: Generalidades. Ciência e conhecimento.

100 a 199: Filosofia e psicologia.

200 a 299: Religião.

300 a 399: Ciências sociais.

400 a 499: Classe vaga. Provisoriamente não ocupada.

500 a 599: Matemática e ciências naturais.

600 a 699: Ciências aplicadas.

700 a 799: Belas artes.

800 a 899: Linguagem. Língua. Linguística.

900 a 999: Geografia. Biografia. História."

In [ ]:
# Classificando o arcevo da biblioteca com .cut()
# o tamanho de rotulos deve ser igual a len(caixas)-1

caixas = [000, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]

rotulos = [
    'Generalidades. Ciência e conhecimento',
    'Filosofia e psicologia',
    'Religião',
    'Ciências sociais',
    'Classe vaga. Provisoriamente não ocupada',
    'Matemática e ciências naturais',
    'Ciências aplicadas',
    'Belas artes',
    'Linguagem. Língua. Linguística',
    'Geografia. Biografia. História.',
]

df['CDU'] = pd.cut(df['localizacao'], bins=caixas, labels=rotulos, right=True, include_lowest=True)

In [ ]:
# A coluna da matricula (“matricula_ou_siape”) não está com um formato muito legível.
# Transforme-a em formato String.

df['matricula_ou_siape'] = df['matricula_ou_siape'].astype('string').str.split('.').str[0]

df.dtypes

In [ ]:
df.head()

In [ ]:
df.to_parquet('../data/processed/emprestimos_exemplares.parquet', index=False)

In [ ]:
df.to_csv('../data/processed/emprestimos_exemplares.csv', encoding='utf-8-sig', index=False)

In [ ]:
df_records_excel_antes_2010 = pd.read_excel(
    '../data/raw/students_records/matricula_alunos.xlsx', sheet_name='Até 2010',skiprows=1
    )

df_records_excel_antes_2010.columns = ['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso']

df_records_excel_antes_2010.columns

In [ ]:
df_records_excel_antes_2010.shape

In [ ]:
df_records_excel_antes_2010['tipo_vinculo_usuario'].unique()

In [ ]:
df_records_excel_apos_2010 = pd.read_excel(
    '../data/raw/students_records/matricula_alunos.xlsx', sheet_name='Após 2010',skiprows=1
    )

df_records_excel_apos_2010.columns = ['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso']

df_records_excel_apos_2010.columns

In [ ]:
df_records_excel_apos_2010.shape

In [ ]:
df_records_excel_apos_2010['tipo_vinculo_usuario'].unique()

In [ ]:
df_records_excel = pd.concat(
    [df_records_excel_antes_2010, df_records_excel_apos_2010],
    ignore_index=True
)

# df_records_excel = df_records_excel.assign(matricula_ou_siape=df_records_excel['matricula_ou_siape'].astype('object'))

df_records_excel.dtypes

In [ ]:
df_records_excel['matricula_ou_siape'] = df_records_excel['matricula_ou_siape'].astype('string').str.split('.').str[0]

In [ ]:
df_records_excel.head()

In [ ]:
df_records_excel.shape

In [ ]:
df_records_excel.to_csv(
    '../data/processed/students_records/matriculas_alunos.csv',
    encoding='utf-8-sig',
    index=False)

In [ ]:
# records do ano de 2018

df_records_2018 = pd.read_csv('../data/raw/students_records/discentes-2018.csv')

# df_records_2018.columns

# matricula, nome_curso, nivel_ensino
df_records_2018.dtypes

In [ ]:
df_records_2018.head()

In [ ]:
df_records_2018.isnull().sum()

In [ ]:
# Corrigindo os nomes dos níveis de ensino com ftfy do df_records_2018
# por exemplo, o df apresenta ESTATÃ\x8dSTICA como nível de ensino, com o ftfy passa a ser ESTÁTISTICA
# assim como com os outros níveis que apresenta o mesmo erro de caracter

df_records_2018['nivel_ensino'] = df_records_2018['nivel_ensino'].apply(ftfy.fix_text)
# df_records_2018['nivel_ensino'].unique()

In [ ]:
# Assim como nos níveis de ensino, os nomes dos cursos também apresenta o mesmo problema

df_records_2018['nome_curso'] = df_records_2018['nome_curso'].apply(
    lambda x: ftfy.fix_text(x) if pd.notnull(x) else x
    )
# df_records_2018['nome_curso'].unique()

In [ ]:
df_records_2018.head()

In [ ]:
# Records do ano de 2019
df_records_2019 = pd.read_csv('../data/raw/students_records/discentes-2019.csv')
df_records_2019.head()

In [ ]:
df_records_2019['nivel_ensino'] = df_records_2019['nivel_ensino'].apply(ftfy.fix_text)
df_records_2019['nivel_ensino'].unique()

In [ ]:
df_records_2019['nome_curso'] = df_records_2019['nome_curso'].apply(
    lambda x: ftfy.fix_text(x) if pd.notnull(x) else x
    )
# df_records_2019['nome_curso'].unique()

In [ ]:
df_records_2018_2019 = pd.concat(
    [df_records_2018, df_records_2019], ignore_index=True
)

In [ ]:
# Renomeando colunas

df_records_2018_2019 = df_records_2018_2019.rename(columns={
    'matricula': 'matricula_ou_siape',
    'nivel_ensino': 'tipo_vinculo_usuario',
    'nome_curso': 'curso'
})

In [ ]:
df_records_2018_2019['matricula_ou_siape'] = df_records_2018_2019['matricula_ou_siape'].astype('string').str.split('.').str[0]

In [ ]:
df_records_2018_2019['tipo_vinculo_usuario'].unique()

In [ ]:
df_records_2018_2019_graduacao = df_records_2018_2019.loc[df_records_2018_2019['tipo_vinculo_usuario'] == 'GRADUAÇÃO', ['matricula_ou_siape','curso','tipo_vinculo_usuario']]

vinculo = {
    'GRADUAÇÃO': 'ALUNO DE GRADUAÇÃO'
}

df_records_2018_2019_graduacao['tipo_vinculo_usuario'] = df_records_2018_2019_graduacao['tipo_vinculo_usuario'].map(vinculo)


df_records_2018_2019_graduacao.head()

In [ ]:
df_records_graduacao = pd.concat(
    [df_records_2018_2019_graduacao, df_records_excel]
)

In [ ]:
df_records_graduacao['matricula_ou_siape'].unique()

In [ ]:
df_records_graduacao.head()

In [ ]:
df_records_json = pd.read_json('../data/raw/students_records/cadastro_alunos.json')

In [ ]:
df_records_json_graduacao = pd.read_json(df_records_json.registros[0])

df_records_json_graduacao['matricula_ou_siape'] = df_records_json_graduacao['matricula_ou_siape'].astype('string').str.split('.').str[0]

df_records_json_graduacao.dtypes

In [ ]:
df_records_json_graduacao.shape

In [ ]:
df_records_json_graduacao.head()

In [ ]:
df_records = pd.concat([
    df_records_graduacao, df_records_json_graduacao
],ignore_index=True)

In [ ]:
df_records.shape

In [ ]:
df_records.dtypes

In [ ]:
df_records['matricula_ou_siape'].unique()

In [ ]:
df_records.head()

In [ ]:
df_records.to_csv(
        '../data/processed/students_records/records.csv',
        index=False, 
        encoding='utf-8-sig'
    )

In [ ]:
df_records.to_parquet('../data/processed/students_records/records.parquet', index=False)

___

PÓS-GRADUAÇÃO

In [ ]:
df = pd.read_parquet('../data/processed/emprestimos_exemplares.parquet')

In [ ]:
df_pos_graduacao = df[df['tipo_vinculo_usuario'] == 'ALUNO DE PÓS-GRADUAÇÃO']

# df_pos_graduacao.dtypes

In [ ]:
df_records_json = pd.read_json('../data/raw/students_records/cadastro_alunos.json')

In [ ]:
df_records_json_pos_graduacao = pd.read_json(df_records_json.registros[1])

df_records_json_pos_graduacao['matricula_ou_siape'] = df_records_json_pos_graduacao['matricula_ou_siape'].astype('string').str.split('.').str[0]

df_records_json_pos_graduacao.dtypes

In [ ]:
emprestimos_pos_graduacao = pd.concat(
    [df_pos_graduacao, df_records_json_pos_graduacao],ignore_index=True
)

In [ ]:
emprestimos_pos_graduacao = df_pos_graduacao.merge(df_records_json_pos_graduacao)

In [ ]:
emprestimos_pos_graduacao

In [ ]:
emprestimos_pos_graduacao.dtypes

In [ ]:
emprestimos_pos_graduacao['data_emprestimo'] = pd.to_datetime(emprestimos_pos_graduacao['data_emprestimo'])
emprestimos_pos_graduacao.dtypes

In [ ]:
emprestimos_pos_graduacao.to_parquet(
        '../data/processed/students_records/records_pos_graduacao.parquet', index=False
    )